In [ ]:
from ultralytics import YOLO
import cv2

model = YOLO(r"E:\best.pt")
results = model.predict(source="0", show=True)
print(results)

In [ ]:
import cv2
import torch
from ultralytics import YOLO
import requests

# Function to send command to ESP32
def send_command(command):
    data = command + '\n'
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# ESP IP URL
url = 'http://192.168.104.99/data'

# URL for the stream
stream_url = "http://100.77.46.130:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Collect results and send labels to ESP32
results_count = 0
while results_count < 10:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model.predict(resized_frame, show=True)

    for result in results:
        for prediction in result:
        # Check if the prediction is not an empty list and contains necessary elements
            if len(prediction) >= 6:  # Check the minimum length required
                label_id = prediction[5]  # Accessing the class ID
                confidence = prediction[4]  # Accessing the confidence score

                # Logic to assign labels based on label IDs

                # Assuming 'Plastic' has label ID 3
                # Assuming 'Biodegradable' has label ID 0
                # Assuming 'Glass' has label ID 1
                # Assuming 'Metal' has label ID 2

                label = '0'  # Default label is 'P' (plastic) for demonstration purposes
                if label_id == 1:
                    label = 'O'
                elif label_id == 0:
                    label = 'B'
                elif label_id == 2:
                    label = 'M'
                elif label_id == 3:
                    label = 'P'

                # Send label to ESP32
                send_command(label)

                # Increment the results count for each detection
                results_count += 1
            else:
                print("Invalid prediction format: ", prediction)

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import torch
import requests
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    data = command + '\n'
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# ESP IP URL
url = 'http://192.168.104.99/data'

# URL for the stream
stream_url = "http://100.77.46.130:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Collect results and send labels to ESP32
results_count = 0
while results_count < 10:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model.predict(resized_frame, show=True)

    for result in results:
        for prediction in result:
            # Check if the prediction is not an empty list and contains necessary elements
            if len(prediction) >= 6:  # Check the minimum length required
                label_id = prediction[5]  # Accessing the class ID

                # Logic to assign labels based on label IDs
                label = 'P'  # Default label is 'P' (plastic) for demonstration purposes
                if label_id == 1:
                    label = 'O'
                elif label_id == 0:
                    label = 'B'
                elif label_id == 2:
                    label = 'M'
                elif label_id == 3:
                    label = 'P'

                # Send label to ESP32
                send_command(label)

                # Increment the results count for each detection
                results_count += 1
            else:
                print("Invalid prediction format: ", prediction)

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()



0: 608x800 1 Plastic, 259.2ms
Speed: 5.2ms preprocess, 259.2ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 800)



Invalid prediction format:  ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Biodegradable', 1: 'Glass', 2: 'Metal', 3: 'Plastic'}
orig_img: array([[[131,  94,  77],
        [131, 103,  84],
        [118,  98,  78],
        ...,
        [ 20,  33,  72],
        [ 24,  38,  74],
        [ 21,  51,  39]],

       [[138, 116,  99],
        [136, 117, 100],
        [122, 108,  90],
        ...,
        [ 20,  33,  72],
        [ 24,  38,  74],
        [ 21,  51,  39]],

       [[191, 187, 175],
        [195, 191, 179],
        [187, 183, 171],
        ...,
        [ 20,  33,  72],
        [ 24,  38,  74],
        [ 21,  51,  39]],

       ...,

       [[136, 118, 198],
        [123, 104, 184],
        [126, 107, 187],
        ...,
        [ 35,  33,  25],
        [ 34,  27,  35],
        [ 39,  21,  63]],

       [[138, 119, 199],
        [125, 106, 186],
        [113,  94, 174],
        ...,


0: 608x800 1 Plastic, 251.0ms
Speed: 5.0ms preprocess, 251.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)



Invalid prediction format:  ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Biodegradable', 1: 'Glass', 2: 'Metal', 3: 'Plastic'}
orig_img: array([[[111,  90,  70],
        [110,  89,  69],
        [120,  99,  79],
        ...,
        [ 24,  37,  76],
        [ 24,  38,  74],
        [ 24,  54,  42]],

       [[105,  89,  70],
        [126, 110,  91],
        [108,  92,  73],
        ...,
        [ 22,  35,  74],
        [ 22,  36,  72],
        [ 22,  52,  40]],

       [[148, 143, 124],
        [162, 157, 138],
        [134, 129, 110],
        ...,
        [ 21,  34,  73],
        [ 21,  35,  71],
        [ 21,  51,  39]],

       ...,

       [[111,  92, 172],
        [130, 111, 191],
        [134, 115, 195],
        ...,
        [ 35,  33,  25],
        [ 37,  30,  38],
        [ 39,  21,  63]],

       [[126, 107, 187],
        [116,  97, 177],
        [133, 114, 194],
        ...,


0: 608x800 1 Plastic, 266.5ms
Speed: 4.8ms preprocess, 266.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)



...

Invalid prediction format:  ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Biodegradable', 1: 'Glass', 2: 'Metal', 3: 'Plastic'}
orig_img: array([[[130, 123, 155],
        [134, 127, 159],
        [132, 125, 157],
        ...,
        [172, 164, 187],
        [172, 163, 188],
        [172, 162, 191]],

       [[130, 123, 155],
        [134, 127, 159],
        [132, 125, 157],
        ...,
        [172, 164, 187],
        [172, 163, 188],
        [172, 162, 191]],

       [[130, 123, 155],
        [134, 127, 159],
        [132, 125, 157],
        ...,
        [172, 164, 187],
        [172, 163, 188],
        [172, 162, 191]],

       ...,

       [[ 81,  69,  65],
        [ 79,  67,  63],
        [ 71,  59,  55],
        ...,
        [ 44,  43,  46],
        [ 28,  28,  27],
        [ 17,  20,   8]],

       [[ 81,  70,  65],
        [ 82,  71,  66],
        [ 77,  66,  61],
        ...,


0: 608x800 1 Metal, 1 Plastic, 252.4ms
Speed: 3.8ms preprocess, 252.4ms inference, 1.3ms postprocess per image at shape (1, 3, 608, 800)



Invalid prediction format:  ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Biodegradable', 1: 'Glass', 2: 'Metal', 3: 'Plastic'}
orig_img: array([[[126, 124, 155],
        [128, 126, 157],
        [130, 128, 159],
        ...,
        [172, 164, 187],
        [172, 164, 187],
        [172, 164, 187]],

       [[126, 124, 155],
        [128, 126, 157],
        [130, 128, 159],
        ...,
        [172, 164, 187],
        [172, 164, 187],
        [172, 164, 187]],

       [[126, 124, 155],
        [128, 126, 157],
        [130, 128, 159],
        ...,
        [172, 164, 187],
        [172, 164, 187],
        [172, 164, 187]],

       ...,

       [[ 77,  65,  64],
        [ 66,  54,  53],
        [ 69,  57,  56],
        ...,
        [ 29,  24,  28],
        [ 15,  11,  11],
        [ 17,  16,   5]],

       [[ 73,  61,  60],
        [ 74,  62,  61],
        [ 85,  73,  72],
        ...,


KeyboardInterrupt: 

: 

In [ ]:
import cv2
from ultralytics import YOLO

# URL for the stream
stream_url = "http://192.168.137.94:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Read and display the video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640,480))

    # Perform object detection on the resized frame
    results = model.predict(resized_frame, show=True)
    print(results)

    # Display the original frame
    cv2.imshow('Frame', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO
model = YOLO(r"E:\best.pt")
model.export(format="saved_model")

In [ ]:
import cv2

# Initialize the camera
cap = cv2.VideoCapture(0)  # Use 0 for the default camera or specify the camera index

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Display the frame
    cv2.imshow("Camera Stream", frame)

    # Check for a key press and break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Exit the loop when 'q' is pressed

# Release the camera and close OpenCV windows
cap.release()
cv2.destroyAllWindows()
